In [1]:
import pandas as pd
from pathlib import Path
import requests

# Plant list

In [2]:
df = pd.read_csv('master_plant_list.csv', index_col=0)
df.shape

(1135, 2)

In [3]:
df.head()

,Family,Species
0,ACANTHACEAE,Carlowrightia arizonica
1,ACANTHACEAE,Justicia californica
2,AIZOACEAE,Mesembryanthemum nodiflorum
3,AIZOACEAE,Trianthema portulacastrum
4,AMARANTHACEAE,Amaranthus crassipes


In [4]:
out_dir = Path("plant_lists")

In [5]:
plantlist = r"C:\Users\matta\Desktop\Documents\Python\clonal_plants\plant_lists\plantlst.txt"
plants = pd.read_csv(plantlist, sep=',')
plants.shape

(93157, 5)

In [6]:
plants.head()

,Symbol,Synonym Symbol,Scientific Name with Author,Common Name,Family
0,ABAB,NaN,Abutilon abutiloides (Jacq.) Garcke ex Hochr.,shrubby Indian mallow,Malvaceae
1,ABAB,ABAM5,Abutilon americanum (L.) Sweet,NaN,NaN
2,ABAB,ABJA,Abutilon jacquinii G. Don,NaN,NaN
3,ABAB,ABLI,Abutilon lignosum (Cav.) G. Don,NaN,NaN
4,ABAB70,NaN,Abietinella abietina (Hedw.) Fleisch.,abietinella moss,Thuidiaceae


In [7]:
names = plants["Scientific Name with Author"].str.split(' ', expand = True)
names = names[0] + ' ' + names[1]
plants['Species'] = names
plants.drop(columns = ['Scientific Name with Author', 'Synonym Symbol', 'Family', 'Common Name'], inplace = True)
plants = plants.drop_duplicates().reset_index(drop=True)

In [18]:
plants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82269 entries, 0 to 82268
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Symbol   82269 non-null  object
 1   Species  82269 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [22]:
df = df.dropna().reset_index(drop=True)
df.shape

(1133, 2)

In [24]:
df.Species.value_counts()

Species
Salix exigua                   1
Carlowrightia arizonica        1
Justicia californica           1
Mesembryanthemum nodiflorum    1
Trianthema portulacastrum      1
                              ..
Allium macropetalum            1
Tidestromia lanuginosa         1
Suaeda nigra                   1
Salsola tragus                 1
Monolepis nuttalliana          1
Name: count, Length: 1133, dtype: int64

In [30]:
plants.Species.value_counts().head()

Species
Astragalus lentiginosus    37
Eriogonum umbellatum       33
Ericameria nauseosa        25
Chrysothamnus nauseosus    25
Lepidium montanum          22
Name: count, dtype: int64

In [39]:
plants[plants.Species == 'Carnegiea gigantea']

,Symbol,Species
12464,CAGI10,Carnegiea gigantea


In [23]:
df.merge(plants, left_on='Species', right_on='Species', how='left')

,Family,Species,Symbol
0,ACANTHACEAE,Carlowrightia arizonica,CAAR7
1,ACANTHACEAE,Justicia californica,JUCA8
2,AIZOACEAE,Mesembryanthemum nodiflorum,MENO2
3,AIZOACEAE,Trianthema portulacastrum,TRPO2
4,AMARANTHACEAE,Amaranthus crassipes,AMCR
...,...,...,...
2153,PTERIDACEAE,Cheilanthes wootoni,NaN
2154,SALICACEAE,Salix exigua,SAEX
2155,SALICACEAE,Salix exigua,SAIN3
2156,SALICACEAE,Salix exigua,SAME2


# Scraping

In [32]:
import requests
from bs4 import BeautifulSoup

In [34]:
url = "https://swbiodiversity.org/seinet/taxa/index.php?taxon=Carnegiea+gigantea"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://swbiodiversity.org/seinet/index.php"
}

session = requests.Session()
session.headers.update(headers)

response = session.get(url, timeout=30)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

print(soup.title.text)
text = soup.get_text(" ", strip=True)

SEINet - AZ/NM Node - Carnegiea gigantea


In [38]:
print(text)

SEINet - AZ/NM Node - Carnegiea gigantea Skip Navigation Sign In SEINet Arizona - New Mexico Chapter ☰ Home Specimen Search Search Collections Map Search Exsiccati Search Images Image Browser Search Images Flora Projects Arizona New Mexico Colorado Plateau Plant Atlas of Arizona (PAPAZ) Sonoran Desert Teaching Checklists Agency Floras NPS - Intermountain USFWS - Region 2 USFS - Southwestern Region BLM Flora Coronado NF Tonto NF Dynamic Floras Dynamic Checklist Dynamic Key Additional Websites New Mexico Flores Plant Atlas Project of Arizona (PAPAZ) Southwest Colorado Wildflowers Vascular Plants of the Gila Wilderness Consortium of Midwest Herbaria Consortium of Southern Rocky Mountain Herbaria Intermountain Region Herbaria Network (IRHN) Mid-Atlantic Herbaria Consortium North American Network of Small Herbaria (NANSH) Northern Great Plains Herbaria Red de Herbarios Mexicanos SERNEC - Southeastern USA Texas Oklahoma Regional Consortium of Herbaria (TORCH) Resources Symbiota Docs Video Tu